In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold 
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.ensemble import AdaBoostClassifier
from matplotlib_venn import venn2


In [ ]:
pd.options.mode.chained_assignment = None  


"""cpt_events = pd.read_csv("CPTEVENTS.csv"
                         ,dtype={'COSTCENTER':object,'CHARTDATE': object,'CPT_NUMBER':object,
                                 'SUBSECTIONHEADER':object})"""

microbiology = pd.read_csv("MICROBIOLOGYEVENTS.csv")
microbiology.drop(['ROW_ID'], axis=1, inplace=True)

##############################################################################################################

drgcodes = pd.read_csv("DRGCODES.csv")
drgcodes.drop(['ROW_ID'], axis=1, inplace=True)

##############################################################################################################

chartevents=pd.read_csv("chartevents_split2.csv")
chartevents.drop(['ROW_ID'], axis=1, inplace=True)

##############################################################################################################

icu_stays=pd.read_csv("ICUSTAYS.csv")
icu_stays.drop(['ROW_ID'], axis=1, inplace=True)

##############################################################################################################
positive_culture=microbiology

positive_culture.dropna(subset =['ORG_ITEMID'],inplace=True)

##############################################################################################################

patients = pd.read_csv('PATIENTS.csv')
patients.drop(['ROW_ID'], axis=1, inplace=True)

##############################################################################################################

admissions= pd.read_csv('ADMISSIONS.csv')
admissions.drop(['ROW_ID'], axis=1, inplace=True)

##############################################################################################################

d_items = pd.read_csv('d_items.csv')
d_items.drop(['ROW_ID'], axis=1, inplace=True)


In [ ]:
#############################################PATIENT & ADMISSION##############################################################


admissions['minadmit']=admissions.groupby('SUBJECT_ID')['ADMITTIME'].transform('min')


# I merging patients and admissions

patient_admission =pd.merge(patients, admissions,how = 'inner',on = ['SUBJECT_ID'])

#converting mindadmit time and DOB into datetime format
patient_admission['minadmit'] = pd.to_datetime(patient_admission['minadmit'], infer_datetime_format=True)
patient_admission['DOB'] = pd.to_datetime(patient_admission['DOB'], infer_datetime_format=True)


# calculating total age through minadmit and date of birth
patient_admission['age'] = list(map(lambda x: x.days , (patient_admission['minadmit'] - patient_admission['DOB'])/365.242 ))

#converting ages < 0 (i.e -285) into 300
patient_admission['age'] = patient_admission['age'].apply(lambda x: 300 if x< 0 else x )


#grouping ages 0: neonate 1:children 2:adult
patient_admission['age_group'] = patient_admission['age'].apply(lambda x: 0 if x<=1 else 
                                                         1 if x>1 and x<18 else 
                                                         2 if x>18 and x<=89 else 
                                                         3 ) 


patient_admission.drop(['DISCHTIME','ADMITTIME','HAS_CHARTEVENTS_DATA','DIAGNOSIS','EDOUTTIME','EDREGTIME','RELIGION','LANGUAGE','DISCHARGE_LOCATION','ADMISSION_LOCATION','ADMISSION_TYPE','minadmit','DEATHTIME','DOB','DOD','DOD_HOSP','DOD_SSN'], axis=1, inplace=True)

patient_admission['sex'] = patient_admission['GENDER'].map({'F': 1, 'M': 0})

patient_admission.drop(['GENDER'], axis=1, inplace=True)



encode_category = LabelEncoder()
patient_admission['marital_encode'] = encode_category.fit_transform(patient_admission['MARITAL_STATUS'].astype(str))

patient_admission['ethnicity_encode'] = encode_category.fit_transform(patient_admission['ETHNICITY'].astype(str))


patient_admission['insurance_encode'] = encode_category.fit_transform(patient_admission['INSURANCE'].astype(str))


patient_admission.drop(['ETHNICITY','MARITAL_STATUS'], axis=1, inplace=True)

##############################################################################################################
